## 알기쉬운 경제이야기

In [ ]:
# Import
import warnings
import os
import pickle
import gc
from dotenv import load_dotenv
from konlpy.tag import Okt
import pandas as pd
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
from langchain.schema import Document
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers import EnsembleRetriever
from langchain_community.retrievers import BM25Retriever
from langchain_teddynote.retrievers import EnsembleRetriever, EnsembleMethod
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_community.chat_models import ChatOpenAI

warnings.filterwarnings('ignore')
load_dotenv()

True

In [3]:
# Path
file_path = './data/corpus-easystory.parquet'
faiss_index_path = './index/easystory_faiss_index.pkl'
bm25_index_path = './index/easystory_bm25_index.pkl'

In [4]:
# Load & Documents
def load_csv_as_documents(file_path):
    df = pd.read_parquet(file_path, engine='pyarrow')
    documents = []
    for _, row in df.iterrows():
        content = row['contents']
        metadata = row['metadata']
        updated_metadata = {
            'doc_id': row['doc_id'],
            'next_id': metadata.get('next_id'),
            'prev_id': metadata.get('prev_id')
        }
        doc = Document(page_content=content, metadata=updated_metadata)
        documents.append(doc)
    return documents

In [5]:
documents = load_csv_as_documents(file_path)
gc.collect()

95

In [6]:
# check 1
for doc in documents[:5]:  
    print(f"Content: {doc.page_content[:100]}...")  
    print(f"Metadata: {doc.metadata}\n")

Content: 대부분의 사람들은 행복하게 살기를 원합니다. 행복한 삶을 이루기 위해 서는 건강한 가운데 가정과 직장 그리고 사회생활에서 적절한 성취를 얻는 것이 무엇보다 중요합니다. 여기에 경제...
Metadata: {'doc_id': '206f6d15-e56f-40de-954c-de9a5eb3abc0', 'next_id': 'e2fe48ac-0088-44c6-9732-5c654d14ef11', 'prev_id': None}

Content: 그렇다면 경제지식이 왜 필요할까요? 과연 이 책을 읽기 위해 투자한 시간의 가치에 상당하는 결실이 돌아올까요? 경제상식이나 지식이 많다고 해서 모든 사 람이 곧 부자가 되고 행복해...
Metadata: {'doc_id': 'e2fe48ac-0088-44c6-9732-5c654d14ef11', 'next_id': None, 'prev_id': '206f6d15-e56f-40de-954c-de9a5eb3abc0'}

Content: 경제상식이나 지식이 우리를 한 순간에 부자로 만들어 주지는 않습니다. 그렇지만 우리가 일상사에서 합리적인 의사결정을 할수 있도록 도와줌으로써 장기적으로 보면 개개인의 행복을 크게 ...
Metadata: {'doc_id': '67a98874-cb4b-42a9-bbcb-ed5ce6f8422a', 'next_id': '500746eb-59d9-4e21-b366-067dea5e55ca', 'prev_id': None}

Content: 경제지식이 단기적으로 주식으로 돈을 버는 기법을 가르쳐 주지는 않습니다. 그렇지만 위험을 분산하고 경제의 흐름을 이해할 수 있도록 함으로써 중장기적으로 손실을 최소화하고 나아가 재...
Metadata: {'doc_id': '500746eb-59d9-4e21-b366-067dea5e55ca', 'next_id': None, 'prev_id': '67a98874-cb4b-42a9-bbcb-ed5ce6f8422a'}

Content: 이러한 경제상식과 지식이 세계경제와

In [7]:
# check 2
print(f"총 {len(documents)}개의 문서가 로드되었습니다.")
print('--------------------------------------')
print(f"첫 번째 문서 내용: {documents[0].page_content[:100]}")
print('--------------------------------------')
print(f"두 번째 문서 내용: {documents[1].page_content[:100]}")
print('--------------------------------------')

총 481개의 문서가 로드되었습니다.
--------------------------------------
첫 번째 문서 내용: 대부분의 사람들은 행복하게 살기를 원합니다. 행복한 삶을 이루기 위해 서는 건강한 가운데 가정과 직장 그리고 사회생활에서 적절한 성취를 얻는 것이 무엇보다 중요합니다. 여기에 경제
--------------------------------------
두 번째 문서 내용: 그렇다면 경제지식이 왜 필요할까요? 과연 이 책을 읽기 위해 투자한 시간의 가치에 상당하는 결실이 돌아올까요? 경제상식이나 지식이 많다고 해서 모든 사 람이 곧 부자가 되고 행복해
--------------------------------------


In [8]:
# okt 한국어 형태소 분석기
okt = Okt()

def okt_tokenize(text):
    return okt.morphs(text)

In [9]:
# Embedding & Indexing
if os.path.exists(faiss_index_path) and os.path.exists(bm25_index_path):
    # 저장된 인덱스를 로드
    with open(faiss_index_path, 'rb') as f:
        faiss_index = pickle.load(f)
    with open(bm25_index_path, 'rb') as f:
        bm25_kkma = pickle.load(f)
else:
    # Embedding 모델 초기화
    embedding_model = HuggingFaceEmbeddings(
        model_name='intfloat/multilingual-e5-large-instruct',
        model_kwargs={'device': 'mps'},
        encode_kwargs={'normalize_embeddings': True},
    )

    # FAISS 벡터스토어 생성 및 저장
    faiss_index = FAISS.from_documents(documents, embedding_model).as_retriever(search_kwargs={"k": 30})
    with open(faiss_index_path, 'wb') as f:
        pickle.dump(faiss_index, f)
    
    # BM25 리트리버 생성 및 저장
    bm25_kkma = BM25Retriever.from_documents(documents, preprocess_func=okt_tokenize)
    bm25_kkma.k = 30
    with open(bm25_index_path, 'wb') as f:
        pickle.dump(bm25_kkma, f)

In [10]:
# Hybrid Search : CC method
weights = [0.96, 0.04]
hybrid_retriever = EnsembleRetriever(
    retrievers=[faiss_index, bm25_kkma],
    weights=weights,
    method=EnsembleMethod.CC
)

In [11]:
# KoReranker
model_path = "Dongjin-kr/ko-reranker"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)

# rerank
def rerank(query, retrieved_documents):
    pairs = [[query, doc.page_content] for doc in retrieved_documents]
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=1024)
    with torch.no_grad():
        scores = model(**inputs, return_dict=True).logits.view(-1).float()
    reranked_docs = sorted(zip(retrieved_documents, scores), key=lambda x: x[1], reverse=True)
    return reranked_docs  # (documents, score) return

# optimize_context
def optimize_context(reranked_docs, max_tokens=8000):
    context = ""
    for doc, score in reranked_docs:
        if len(context) + len(doc.page_content) > max_tokens:
            break
        context += doc.page_content + "\n\n"
    return context.strip()

In [12]:
# Prompt
prompt_template = PromptTemplate(
    input_variables=["query", "retrieved_contents"],
    template="""
당신은 경제 금융 전문가 '정수빈'입니다.
블로그 시리즈의 글을 작성한다고 생각해주세요.
이 블로그는 경제 지식이 없거나 경제 개념을 쉽게 배우고 싶은 사람들을 대상으로 합니다.
당신의 역할은 경제 용어에 대해 친절하고 쉽게 이해할 수 있는 설명을 제공하는 블로그 글을 작성하는 것입니다.

블로그 글 초반부에서는 인사말을 반드시 작성해야 합니다. 
'안녕하세요, 독자님들~ 수빈이입니다! 오늘도 저와 함께 쉽게 경제 공부를 해볼까요?' 등의 말로 시작해야 합니다.

그 다음으로 개념을 상세하게 설명해 주세요. 
그리고 쉬운 예시를 3개 만들어서 동화처럼 설명해 주세요.
실생활에서 접할 수 있는 다양한 상황을 포함하도록 합니다.

다음 단계로 이 용어를 이해하는 것이 왜 중요한지 요약해 주세요.

글 마무리 문구도 작성해야 합니다.
'오늘의 경제 공부는 어떠셨나요? 제 설명이 여러분께 도움이 되셨으면 좋겠어요. 오늘도 방문해주셔서 감사합니다 ^_^' 등의 말로 마무리해야 합니다.

경제 지식이 전혀 없는 사람도 쉽게 이해하고 흥미롭게 읽을 수 있도록 친근하고 쉽게 작성해 주세요.

# 주의사항:
1. 문서의 내용을 기반으로만 글을 작성하세요. 내용을 지어내거나 사실과 다르게 작성하지 마세요.
2. 문서에는 관련 메타데이터가 포함되어 있습니다. 메타데이터와 본문 내용을 모두 고려하여 정확한 답변을 제공하세요.
3. 만약 설명할 수 없는 부분이 있다면, '모르겠습니다'라고 답하세요.
4. 모든 제목은 #나 ## 같은 Markdown 표시 없이 굵은 글씨(**)로 나타나야 합니다. 예를 들어'## 경기가 무엇인가요?'는 '**경기가 무엇인가요?**'로 표시합니다.
5. 본문과 인사말에서는 일반 텍스트 형식을 사용하고, 필요할 경우 단어에만 굵은 글씨를 사용해주세요.
6. 글은 최대한 길게 작성해 주세요.
7. 글 초반부 인사와 마무리 인사는 길고 다채롭게 표현하면 좋습니다.
8. 글 초반부 인사 : 약 500자, 마무리 인사 : 약 500자 길이로 작성해 주세요.

이제 주제에 맞게 블로그 글을 작성해 주세요.
질문: {query}
단락: {retrieved_contents}
답변:
"""
)

In [13]:
# LLM & Chain
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3)
chain = LLMChain(llm=llm, prompt=prompt_template)

In [14]:
def generate_answer_with_debug(query, top_k_retrieve=30, top_k_rerank=5):
    # Step 1: Retrieve documents
    retrieved_documents = hybrid_retriever.invoke(query)
    
    # 상위 top_k_retrieve 문서만 유지
    unique_documents = []
    seen = set()
    for doc in retrieved_documents:
        if doc.page_content not in seen:
            unique_documents.append(doc)
            seen.add(doc.page_content)
        if len(unique_documents) == top_k_retrieve:
            break
    
    print("최종 Retrieve 단계에서 검색된 문서:")
    print("--------------------------------------------------")
    for i, doc in enumerate(unique_documents, 1):
        print(f"문서 {i}:\n내용: {doc.page_content[:100]}...\n")

    # Step 2: Rerank documents
    reranked_documents = rerank(query, unique_documents)
    reranked_documents = reranked_documents[:top_k_rerank]
    print("\nReranker를 통해 재정렬된 문서:")
    print("--------------------------------------------------")
    for i, (doc, score) in enumerate(reranked_documents, 1):
        print(f"문서 {i} (점수: {score}):\n내용: {doc.page_content[:100]}...\n")

    # Step 3: 최적화된 컨텍스트 생성
    reranked_contents = optimize_context(reranked_documents)
    
    # Step 4: Generate response using LLM
    response = chain.run({"query": query, "retrieved_contents": reranked_contents})
    return response

In [15]:
query = "환율이 매일 변동하는 이유는 무엇인가요?"
response = generate_answer_with_debug(query)

# Result
print("질문:", query)
print("답변:", response)

python(73585) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


최종 Retrieve 단계에서 검색된 문서:
--------------------------------------------------
문서 1:
내용: 실제 사용한 날짜와 결제 날짜가 차이가 있으므로 더 나은 환율로 결제할 수 있기 때문입니다. 국외 송금할 경우와 국외에서 신용카드를 사용하였을 경우에 적용되는 환율은 전신환매매율입...

문서 2:
내용: 환율제도는 나라마다 그 결정방식이 다른데 크게 고정환율제도와 변동환율제도로 나누어집니다. 고정환율제도는 정부 또는 중앙은행이 외환시장에 개입하여 환율을 일정한 수준으로 유지시키는 ...

문서 3:
내용: 환율은 수출입기업을 비롯한 경제주체들의 외국과 거래에 큰 영향을 미치기 때문에 많은 사람들은 환율변동에 큰 관심을 보입니다. 이제 환 율 변동이 나라경제에 어떤 영향을 미치는지 알...

문서 4:
내용: 그러면 환율은 어디서 어떻게 결정될까요? 식품이나 TV와 같은 상품의 가격이 시장에서 정해지듯이, 돈의 대외 가치인 환율은 외환이 거래되는 시장에서 외환의 수요와 공급에 의해 결정...

문서 5:
내용: 즉 환율안정을 위해 정책 금리를 자주 변경하게 되면 금리의 변동성 확대로 금융시장이 더욱 불안정해지는 문제가 발생하여 득보다 실이 더 클 수도 있습니다....

문서 6:
내용: 환율이 오르내리는 이유는 상품시장에서의 균형가격 결정 원리와 동일합니다. 즉 환율은 외환시장에서 외환의 수요와 공급에 따라 결정되므로 해당 화폐에 대한 수요가 커지면 그 화폐의 가...

문서 7:
내용: 그러한 이유로 단기적인 환율 음직임을 예측하는 것은 매우 어렵습니다. 더군다나 전문가가 아닌 일반인으 로서는 미래의 환율 움직임을 예측하기 더더욱 어렵습니다....

문서 8:
내용: 물론 여러 가지 이유로 실제로 이렇게 정확하게 맞아 떨어지지는 않습니다. 여기서 환율을 1,000 원/달러(1달러당 1,000원)로 표시한 것은 국제 금융거래에 있어 기축통화인 미...

문서 9:
내용: 또한 환율변동은 장기적으로 국가 간의 